# Hyperparameter Tuning in Tourism - Preprocessing

**[Pier Paolo Ippolito](https://www.linkedin.com/in/pierpaolo28/)**

In this example are taken the 3 datasets from [2015 Flight Delays and Cancellations](https://www.kaggle.com/usdot/flight-delays) and processed in order to create a dataset to perform a regression analysis to predict the delay at the arrival of the flight. The dataset generated in this notebook is then going to be used in the "Hyperparameter Tuning - Analysis" notebook for further modelling analysis.

![](jerry-zhang-unsplash.jpg)

<span>Photo by <a href="https://unsplash.com/@z734923105?utm_source=unsplash&amp;utm_medium=referral&amp;utm_content=creditCopyText">Jerry Zhang</a> on <a href="https://unsplash.com/s/photos/flight?utm_source=unsplash&amp;utm_medium=referral&amp;utm_content=creditCopyText">Unsplash</a></span>

## Preprocessing

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv('/kaggle/input/flight-delays/flights.csv', low_memory=False)
df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


Deleting first all the rows which have NaNs (Not a number) for the Arrival Delay column (our target variable). Successively, dropping all the comluns which have NaNs.

In [3]:
df = df[df['ARRIVAL_DELAY'].notna()]
df = df.loc[:, ~df.isnull().any(axis=0)]
df.columns

Index(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER',
       'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT',
       'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME',
       'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED'],
      dtype='object')

In [4]:
df.isnull().sum()

YEAR                   0
MONTH                  0
DAY                    0
DAY_OF_WEEK            0
AIRLINE                0
FLIGHT_NUMBER          0
TAIL_NUMBER            0
ORIGIN_AIRPORT         0
DESTINATION_AIRPORT    0
SCHEDULED_DEPARTURE    0
DEPARTURE_TIME         0
DEPARTURE_DELAY        0
TAXI_OUT               0
WHEELS_OFF             0
SCHEDULED_TIME         0
ELAPSED_TIME           0
AIR_TIME               0
DISTANCE               0
WHEELS_ON              0
TAXI_IN                0
SCHEDULED_ARRIVAL      0
ARRIVAL_TIME           0
ARRIVAL_DELAY          0
DIVERTED               0
CANCELLED              0
dtype: int64

In [5]:
airports= pd.read_csv('/kaggle/input/flight-delays/airports.csv')
airports.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [6]:
airports.shape

(322, 7)

Checking which rows have NaNs

In [7]:
airports[airports.isna().any(axis=1)]

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
96,ECP,Northwest Florida Beaches International Airport,Panama City,FL,USA,NaN,NaN
234,PBG,Plattsburgh International Airport,Plattsburgh,NY,USA,NaN,NaN
313,UST,Northeast Florida Regional Airport (St. August...,St. Augustine,FL,USA,NaN,NaN


In [8]:
airports = airports.dropna()
airports.shape

(319, 7)

In [9]:
airlines = pd.read_csv('/kaggle/input/flight-delays/airlines.csv')
airlines.head()

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways


Merging the 3 different datasets in order to create our final table and dropping unnecessary columns.

In [10]:
df = df.merge(airlines, left_on='AIRLINE', right_on='IATA_CODE', how='inner')
df = df.drop(['AIRLINE_x','IATA_CODE'], axis=1)
df = df.rename(columns={"AIRLINE_y":"AIRLINE"})
df = df.merge(airports, left_on='ORIGIN_AIRPORT', right_on='IATA_CODE', how='inner')
df = df.merge(airports, left_on='DESTINATION_AIRPORT', right_on='IATA_CODE', how='inner')
df = df.drop(['LATITUDE_x', 'LONGITUDE_x', 'STATE_y', 'COUNTRY_y', 'LATITUDE_y', 'LONGITUDE_y','STATE_x', 'COUNTRY_x', 'IATA_CODE_x','IATA_CODE_y', 'YEAR',
              'AIRPORT_x', 'AIRPORT_y', 'CITY_x', 'CITY_y'], axis=1)
df.head()

,MONTH,DAY,DAY_OF_WEEK,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,...,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,AIRLINE
0,1,1,4,98,N407AS,ANC,SEA,5,2354.0,-11.0,...,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,Alaska Airlines Inc.
1,1,1,4,108,N309AS,ANC,SEA,45,41.0,-4.0,...,173.0,1448,451.0,4.0,509,455.0,-14.0,0,0,Alaska Airlines Inc.
2,1,1,4,134,N464AS,ANC,SEA,155,140.0,-15.0,...,170.0,1448,547.0,11.0,633,558.0,-35.0,0,0,Alaska Airlines Inc.
3,1,1,4,114,N303AS,ANC,SEA,220,209.0,-11.0,...,176.0,1448,620.0,8.0,640,628.0,-12.0,0,0,Alaska Airlines Inc.
4,1,1,4,730,N423AS,ANC,SEA,505,457.0,-8.0,...,179.0,1448,912.0,4.0,930,916.0,-14.0,0,0,Alaska Airlines Inc.


In [11]:
df.shape

(5222000, 24)

Checking if any column has NaNs

In [12]:
df.isnull().sum()

MONTH                  0
DAY                    0
DAY_OF_WEEK            0
FLIGHT_NUMBER          0
TAIL_NUMBER            0
ORIGIN_AIRPORT         0
DESTINATION_AIRPORT    0
SCHEDULED_DEPARTURE    0
DEPARTURE_TIME         0
DEPARTURE_DELAY        0
TAXI_OUT               0
WHEELS_OFF             0
SCHEDULED_TIME         0
ELAPSED_TIME           0
AIR_TIME               0
DISTANCE               0
WHEELS_ON              0
TAXI_IN                0
SCHEDULED_ARRIVAL      0
ARRIVAL_TIME           0
ARRIVAL_DELAY          0
DIVERTED               0
CANCELLED              0
AIRLINE                0
dtype: int64

Checking how many distinct values each column has

In [13]:
df.nunique()

MONTH                    11
DAY                      31
DAY_OF_WEEK               7
FLIGHT_NUMBER          6946
TAIL_NUMBER            4895
ORIGIN_AIRPORT          319
DESTINATION_AIRPORT     319
SCHEDULED_DEPARTURE    1316
DEPARTURE_TIME         1440
DEPARTURE_DELAY        1192
TAXI_OUT                184
WHEELS_OFF             1440
SCHEDULED_TIME          543
ELAPSED_TIME            711
AIR_TIME                675
DISTANCE               1346
WHEELS_ON              1440
TAXI_IN                 184
SCHEDULED_ARRIVAL      1433
ARRIVAL_TIME           1440
ARRIVAL_DELAY          1221
DIVERTED                  1
CANCELLED                 1
AIRLINE                  14
dtype: int64

In [14]:
df.to_csv('flight_data.csv')

## Machine Learning

Performing a basic Machine Learning analysis in order to test the quality of the generated data.

Transforming categorical columns into numeric equivalents

In [15]:
le = LabelEncoder()
df['TAIL_NUMBER']= le.fit_transform(df['TAIL_NUMBER'])
df['ORIGIN_AIRPORT']= le.fit_transform(df['ORIGIN_AIRPORT'])
df['DESTINATION_AIRPORT']= le.fit_transform(df['DESTINATION_AIRPORT'])
df['ORIGIN_AIRPORT']= le.fit_transform(df['ORIGIN_AIRPORT'])
df['AIRLINE']= le.fit_transform(df['AIRLINE'])

Dividing our dataset into features/labels and training/test partitions

In [16]:
X = df.drop(['ARRIVAL_DELAY'], axis=1)
X = pd.get_dummies(X, prefix_sep='_')
X = StandardScaler().fit_transform(X)
Y = df['ARRIVAL_DELAY']

X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, test_size = 0.30, random_state = 101)

In [17]:
trained = Lasso().fit(X_Train,Y_Train)
predictions = trained.predict(X_Test)
mean_squared_error(Y_Test, predictions)

97.51762370166568